
## Data from PokeAPI

### Start by setting some reusable variables

In [1]:
import requests, json, os, collections
# import pokebase as pb

baseUrl = "https://pokeapi.co/api/v2"

total = (requests.get(f"{baseUrl}/pokemon-species?limit=1").json())["count"]

print(f"There are currently {total} Pokemon species")

limit = 999

There are currently 898 Pokemon species


### Pull out and store all the Pokemon data we need

This way, we can use the data later without calling the API again (avoid rate limiting)

In [2]:
# recursive function to pull out all values from a nested dictionary
def get_all_values(nested_dictionary):
    value_list = []
    for key, value in nested_dictionary.items():
        if type(value) is dict:
            value_list.extend(get_all_values(value))
        elif type(value) is str:
            value_list.append(value)
    return value_list

In [3]:
# Function to pull all out and store all necessary pokemon data
def get_pokemon_data(baseUrl, limit):
    pokemon_species = requests.get(f"{baseUrl}/pokemon-species?limit={limit}").json()

    pokemon_data = {}

    for poke in pokemon_species["results"]:
        species_response = requests.get(poke["url"]).json()
        dex_num = species_response["id"]
        # print(dex_num)
        
        pokemon_response = requests.get(f"{baseUrl}/pokemon/{dex_num}").json()

        ### grabbing all sprite images from the API response ###
        sprites = get_all_values(pokemon_response["sprites"]) # need to recurse through the sprite objects to pull out all the image urls

        ### grabbing the typings from the API response
        typing = []
        for poke_type in pokemon_response["types"]:
            typing.append(poke_type["type"]["name"])

        if (dex_num < 10):
            dex_num = '00' + str(dex_num)
        elif (dex_num < 100):
            dex_num = '0' + str(dex_num)
        else: dex_num = str(dex_num)

        # try to get color name
        try:
            color = species_response["color"]["name"]
        except Exception as e:
            color = None

        # try to get shape name
        try:
            shape = species_response["shape"]["name"]
        except Exception as e:
            shape = None

        ## set up Pokemon object
        pokemon = {
            "name": species_response["name"],
            "color": color,
            "shape": shape,
            "sprites": sprites,
            "types": typing
        }

        pokemon_data[dex_num] = pokemon

    items = sorted(pokemon_data.items()) # sort the pokemon by dex number
    ordered_data = collections.OrderedDict(items)

    return ordered_data

In [4]:
# Gather the data and place it into a JSON for use later
get_new_data = True

if os.path.exists("./data/detailed_pokemon_species.json") and get_new_data == False:
    detailed_pokemon_data = json.load(open("./data/detailed_pokemon_species.json"))
else:
    detailed_pokemon_data = get_pokemon_data(baseUrl, limit)
    # save the data to disk
    prefix = os.environ.get('poke-prefix', './')
    with open(f"{prefix}data/detailed_pokemon_species.json", "w") as f:
        f.write(json.dumps(detailed_pokemon_data))